In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TemplateHaskell #-}

import AdventOfCode
import Control.Lens
import Data.Function.Memoize
import Data.Functor.Identity

import qualified Data.Attoparsec.ByteString.Char8 as A
import qualified Data.Map.Strict as M
import qualified Data.Vector as V

input <- dayLines 23

In [2]:
type Reg = Char
type ValueReg = Either Int Reg

parseReg = A.satisfy A.isAlpha_ascii
parseValue = A.signed A.decimal

parseValueReg :: A.Parser ValueReg
parseValueReg = A.eitherP parseValue parseReg

In [3]:
data Instr 
    = Set Reg ValueReg
    | Sub Reg ValueReg
    | Mul Reg ValueReg
    | Jnz ValueReg ValueReg
    deriving (Eq, Show)

parseInstr = A.choice
    [ Set <$> ("set" *> pR)  <*> pVR
    , Sub <$> ("sub" *> pR)  <*> pVR
    , Mul <$> ("mul" *> pR)  <*> pVR
    , Jnz <$> ("jnz" *> pVR) <*> pVR
    ]
    where
        pVR = A.space *> parseValueReg
        pR  = A.space *> parseReg

registerMap = M.fromList $ zip ['a'..'p'] (repeat 0)
instrVec    = V.fromList $ map (parsed parseInstr) input

In [4]:
data MachineState = MachineState
    { _registers :: M.Map Char Int
    , _counter   :: Int
    } deriving (Eq, Show)

makeLenses ''MachineState

getValueReg :: M.Map Char Int -> ValueReg -> Int
getValueReg registers = either id (\r -> registers ^. at r . non 0)

incr = counter +~ 1

executeInstr :: Instr -> MachineState -> MachineState
executeInstr instr ms = ms & case instr of
    Set reg vr -> incr . (registers %~ (\rs -> rs & at reg ?~          (getValueReg rs vr)))
    Sub reg vr -> incr . (registers %~ (\rs -> rs & ix reg %~ (subtract(getValueReg rs vr))))
    Mul reg vr -> incr . (registers %~ (\rs -> rs & ix reg %~        (*(getValueReg rs vr))))
    Jnz vr vr' -> if getValueReg (ms ^. registers) vr /= 0
        then counter %~ (+ (getValueReg (ms ^. registers) vr'))
        else incr

In [5]:
loop :: V.Vector Instr -> MachineState -> Int -> Int
loop instrs ms mulCount = case instrs ^? ix (ms ^. counter) of
    Nothing       -> mulCount
    Just (Mul reg vr) -> loop instrs (executeInstr (Mul reg vr) ms) (mulCount + 1)
    Just curr     -> loop instrs (executeInstr curr ms) mulCount

In [6]:
loop instrVec (MachineState registerMap 0) 0

4225

In [9]:
part2 :: Int
part2 = let
    b = 106700
    in runIdentity $ loopBody (b,2,2,1,0)
    where
        c = 123700
        loopBody (b,d,e,f,h) = do
            f <- pure $ if d*e == b then 0 else f
            if (e+1) /= b
            then loopBody (b,d,e+1,f,h)
            else
                if (d+1) /= b
                then loopBody (b,d+1,2,f,h)
                else
                    if f == 0
                    then returner (b,h+1)
                    else returner (b,h)
        returner (b,h) = if b==c
                then return h
                else loopBody (b+17,2,2,1,h)

In [13]:
candidates = [106700,(106700+17)..123700]

prime :: Int -> Bool
prime = memoize $ \n -> case n of
    2 -> True
    n -> let s = fromIntegral (floor (sqrt (fromIntegral n))) in null [f | f <- filter prime [2..s], n `rem` f == 0]

length $ filter (not . prime) candidates

905